In [13]:
from google.colab import auth
auth.authenticate_user()

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
%cd /content/drive/MyDrive/Final-Project-snapAddy

/content/drive/MyDrive/Final-Project-snapAddy


In [22]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd() / "src"))

In [23]:
!ls

archive  data  data_prep_eda.ipynb  results_comparison.ipynb  src


In [24]:
import pandas as pd

from utils.data_preparation_pipeline import data_preparation_pipeline
from utils.eval_utils import evaluate_predictions
from utils.text_utils import normalize_text

ModuleNotFoundError: No module named 'utils.text_utils'

## Description Model Approach 1: Baseline

In [ ]:
X, y_dep, y_sen, meta = data_preparation_pipeline(
    jobs_annotated_df,
    extension=False,
    annotated=True
)

X_norm = X.apply(normalize_text)

In [ ]:
result = {
    "model": "Rule-based",
    "extension": False,
    "target": "department",
    "accuracy": metrics["accuracy"],
    "macro_f1": metrics["macro_f1"]
}

In [ ]:
import pandas as pd
from pathlib import Path

results_path = Path("../results")
results_path.mkdir(exist_ok=True)

pd.DataFrame([result]).to_csv(
    results_path / "baseline_rule_based.csv",
    index=False
)